In [8]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]


In [3]:
train_path = 'Dataset/Train'
valid_path = 'Dataset/Test'


In [4]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)



In [5]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

In [6]:
  # useful for getting number of classes
folders = glob('Dataset/Train/*')
  

In [9]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
x = Dense(64, activation='relu')(x)
prediction = Dense(2, activation='softmax')(x)


In [10]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)


In [11]:
# view the structure of the model
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [12]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Dataset/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Dataset/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')



Found 290 images belonging to 2 classes.
Found 70 images belonging to 2 classes.


In [14]:
# fit the model
r= model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=10,
  validation_steps=8
)

Epoch 1/5
10/10 [==============================] - 318s 32s/step - loss: 1.8720 - accuracy: 0.4586 - val_loss: 0.5612 - val_accuracy: 0.5882
Epoch 2/5
10/10 [==============================] - 265s 26s/step - loss: 0.5030 - accuracy: 0.7034 - val_loss: 0.2303 - val_accuracy: 1.0000
Epoch 3/5
10/10 [==============================] - 275s 28s/step - loss: 0.2044 - accuracy: 0.9724 - val_loss: 0.0713 - val_accuracy: 0.9551
Epoch 4/5
10/10 [==============================] - 293s 29s/step - loss: 0.1021 - accuracy: 0.9897 - val_loss: 0.0852 - val_accuracy: 1.0000
Epoch 5/5
10/10 [==============================] - 282s 28s/step - loss: 0.0598 - accuracy: 1.0000 - val_loss: 0.0858 - val_accuracy: 1.0000


In [ ]:

r.save('VGG16DRAFT.h5')
